In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from ipywidgets import interactive,FloatSlider
import matplotlib

In [2]:
def sigmoid(x,x_0,L,y_0,k):
    return (x-x_0)+(L/2)+y_0 - L/(np.exp(-k*(x-x_0))+1)


def speed_sigmoid_func(x,x_0a,x_0b,L,k,y_0):
    output = np.zeros_like(x)
    output[(x>=x_0a)&(x<=x_0b)] = y_0
    output[x<x_0a] = sigmoid(x[x<x_0a],x_0a,L,y_0,k)
    output[x>x_0b] = sigmoid(x[x>x_0b],x_0b,L,y_0,k)
    return output


In [4]:
def f(x_0a,x_0b,L,k,y_0):
    yl,yu = -4,8
    inputs = np.linspace(-5,8,100)
    fig = plt.figure(figsize=(12,8))
    ax = plt.subplot()
    plt.plot(inputs,speed_sigmoid_func(inputs,x_0a,x_0b,L,k,y_0))
    plt.ylim([yl,yu])
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')

    # Eliminate upper and right axes
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')

    # Show ticks in the left and lower axes only
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    
    ax.spines['bottom'].set_position('zero')
    ax.spines['left'].set_position('zero')

    
def slider(start,stop,step,init):#,init):
    return FloatSlider(
    value=init,
    min=start,
    max=stop,
    step=step,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
    
interactive_plot = interactive(f, x_0a =slider(-2,6,0.1,-0.4),x_0b =slider(-2,6,0.1,3.6),
                                L=slider(0,4,0.1,0.8),k=slider(.1,10,0.01,4.),y_0=slider(0,4,0.1,1.6))
output = interactive_plot.children[-1]
output.layout.height = '1000px'
interactive_plot


interactive(children=(FloatSlider(value=-0.4, continuous_update=False, description='x_0a', max=6.0, min=-2.0),…

**Will's explanation of the perfect PID controller for windspeed/groundspeed.**

Start with the force equation:
$$m\dot{v} = -c(v+w)$$

where $v$ is the fly's groundspeed, and $w$, the wind speed (along the fly body axis) is positive when the wind is blowing against the fly's direct.

Before we consider the force the fly exerts, the force the fly experiences (right side) is a constant $c$ times the sum of the groundspeed and the windspeed.

For example in the case where the groundspeed $v=1$ and the windspeed is $-1$ (wind going with the fly), the force is 0. If $v=1$ and $w=1$ (wind going against the fly), the fly is experiencing a force of 2.

Then add in the force (thrust) the fly can produce:

$$m\dot{v} = -c(v+w) + F(v,v_{sp})$$

$F$ is some function, $v$ is the current fly speed, $v_{sp}$ is the set point velocity the fly wants.

If we set the acceleration in the above to 0, 

$$c(v+w) = F(v,v_{sp})$$
$$ v = \frac{F}{c} - w $$

If we plot groundspeed as a function of windspeed, the system described above will look like this:

<img src="files/simple_airspeed_controller.JPG" width=400px></center>

there are range of wind values for which the fly's thrust can completely compensate for the wind and achieve equilibrium $\dot{v} = 0$.

$w_1$ is the maximum postive (into the fly) wind velocity for which the fly can produce a fully compensating counter-force (call this $F_{max}$) into the wind. After this point, the sum of forces becomes negative and so then does $\dot{v}$. (why is it linear with respect to $w$?)

As we head towards $w_2$, the thrust decreases and could become negative, ie, the fly is applying force backwards to stop from being pushed forwards (negative w) by the wind.

At $w_2$, we have the largest backward force the fly can produce in the face of a negative wind (wind going in direction of the fly), after which point the fly starts getting pushed forward.

In [8]:
def pre_sigmoid(x,x_0,L,y_0,k):
     return (L/2)+y_0 - L/(np.exp(-k*(x-x_0))+1)


def sigmoid(x,x_0,L,y_0,k,m):
    return m*(x-x_0)+(L/2)+y_0 - L/(np.exp(-k*(x-x_0))+1)


def speed_sigmoid_func(x,x_0a,x_0b,L,k,y_0,m):
    output = np.zeros_like(x)
    output[(x>=x_0a)&(x<=x_0b)] = y_0
    output[x<x_0a] = sigmoid(x[x<x_0a],x_0a,L,y_0,k,m)
    output[x>x_0b] = sigmoid(x[x>x_0b],x_0b,L,y_0,k,m)
    return output

def f(x_0a,x_0b,L,k,y_0,m):
    yl,yu = -4,8
    inputs = np.linspace(-5,8,100)
    fig = plt.figure(figsize=(12,8))
    plt.subplot(2,1,1)
    plt.plot(inputs,pre_sigmoid(inputs,x_0a,L,y_0,k))
    plt.plot(inputs,y_0*np.ones_like(inputs),'--')
    plt.ylim([yl,yu])
    ax= plt.subplot(2,1,2)
    plt.plot(inputs,sigmoid(inputs,x_0a,L,y_0,k,m))
    plt.plot(inputs,sigmoid(inputs,x_0b,L,y_0,k,m))
    plt.plot(inputs,speed_sigmoid_func(inputs,x_0a,x_0b,L,k,y_0,m),label='final curve',color='blue')
    plt.plot(inputs,m*(inputs-x_0a)+(L/2)+y_0,'--')
    plt.plot(inputs,m*(inputs-x_0b)-(L/2)+y_0,'--')
    
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')

    # Eliminate upper and right axes
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')

    # Show ticks in the left and lower axes only
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    
    ax.spines['bottom'].set_position('zero')
    ax.spines['left'].set_position('zero')


    
    plt.ylim([yl,yu])
    plt.legend()
    
def slider(start,stop,step,init):#,init):
    return FloatSlider(
    value=init,
    min=start,
    max=stop,
    step=step,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
    
interactive_plot = interactive(f, x_0a =slider(-2,6,0.01,-0.4),x_0b =slider(-2,6,0.01,3.6),
                                L=slider(0,4,0.1,0.8),k=slider(.1,10,0.01,4.),y_0=slider(0,4,0.1,1.6),
                                  m=slider(0,4,0.1,1.))
output = interactive_plot.children[-1]
output.layout.height = '600px'
interactive_plot


interactive(children=(FloatSlider(value=-0.4, continuous_update=False, description='x_0a', max=6.0, min=-2.0, …

In [9]:

def f(x_0a,x_0b,L,k,y_0,m,theta):
    yl,yu = -4,8
    inputs = np.linspace(-5,8,100)
    fig = plt.figure(figsize=(12,8))
    ax= plt.subplot(1,3,1)
    ax.set_aspect('equal')
    plt.plot(inputs,pre_sigmoid(inputs,x_0a,L,y_0,k))
    plt.plot(inputs,y_0*np.ones_like(inputs),'--')
    plt.ylim([yl,yu])
    ax = plt.subplot(1,3,2)
    ax.set_aspect('equal')
    plt.plot(inputs,sigmoid(inputs,x_0a,L,y_0,k,m))
    plt.plot(inputs,sigmoid(inputs,x_0b,L,y_0,k,m))
    outputs = speed_sigmoid_func(inputs,x_0a,x_0b,L,k,y_0,m)
    plt.plot(inputs,m*(inputs-x_0a)+(L/2)+y_0,'--')
    plt.plot(inputs,m*(inputs-x_0b)-(L/2)+y_0,'--')
    plt.plot(inputs,outputs,label='final curve',color='blue')
    plt.ylim([yl,yu])
    xlim = ax.get_xlim()
    plt.legend()
    
    rot_mat = np.array([[np.cos(theta),-1.*np.sin(theta)],[np.sin(theta),np.cos(theta)]])
    rotation_origin = np.array([x_0a+(x_0b-x_0a)/2,y_0])
    plt.plot(rotation_origin[0],rotation_origin[1],'o',color='r')
    rotation_origin_ones = np.repeat(rotation_origin[:,None],100,axis=1)
    inputs1,outputs1 = np.dot(rot_mat,np.vstack((inputs,outputs))-rotation_origin_ones)+rotation_origin_ones
    ax = plt.subplot(1,3,3)
    ax.set_aspect('equal')
    plt.plot(inputs,outputs,color='blue')
    plt.plot(inputs1,outputs1,label='rotated curve')
    plt.ylim([yl,yu])
    plt.xlim(xlim)
    plt.legend()
    

interactive_plot = interactive(f, x_0a =slider(-2,6,0.01,-0.4),x_0b =slider(-2,6,0.01,3.6),
                                L=slider(0,4,0.1,0.8),k=slider(.1,10,0.01,4.),y_0=slider(0,4,0.1,1.6),
                                  m=slider(0,4,0.1,1.),theta=slider(0,np.pi/2,0.1,np.pi/6))
    
    
output = interactive_plot.children[-1]
output.layout.height = '300px'
interactive_plot


interactive(children=(FloatSlider(value=-0.4, continuous_update=False, description='x_0a', max=6.0, min=-2.0, …

In [12]:
#Concept-proofing the rotation input-output
def find_nearest(array, value):
    #For each element in value, returns the index of array it is closest to.
    #array should be 1 x n and value should be m x 1
    idx = (np.abs(array - value)).argmin(axis=1) #this rounds up and down (
    #of the two values in array closest to value, picks the closer. (not the larger or the smaller)
    return idx


def f(x_0a,x_0b,L,k,y_0,m,theta):
    yl,yu = -4,8
    buffer = 10
    num_points = 1000
    inputs = np.linspace(yl-buffer,yu+buffer,num_points)
    outputs = speed_sigmoid_func(inputs,x_0a,x_0b,L,k,y_0,m)
    fig = plt.figure(figsize=(12,8))
    rot_mat = np.array([[np.cos(theta),-1.*np.sin(theta)],[np.sin(theta),np.cos(theta)]])
    rotation_origin = np.array([x_0a+(x_0b-x_0a)/2,y_0])
    plt.plot(rotation_origin[0],rotation_origin[1],'o',color='r')
    rotation_origin_ones = np.repeat(rotation_origin[:,None],num_points,axis=1)
    inputs1,outputs1 = np.dot(rot_mat,np.vstack((inputs,outputs))-rotation_origin_ones)+rotation_origin_ones
    ax = plt.subplot()
    ax.set_aspect('equal')
    plt.plot(inputs,outputs,color='blue')
    plt.plot(inputs1,outputs1,label='rotated curve')
    
    which_inputs = find_nearest(inputs1,inputs[:,None])
    
    plt.plot(inputs,outputs1[which_inputs],'o',color='orange')
    
    plt.ylim([yl,yu])
#     plt.xlim(xlim)
    plt.legend()
    

interactive_plot = interactive(f, x_0a =slider(-2,6,0.1,-0.4),x_0b =slider(-2,6,0.1,3.6),
                                L=slider(0,4,0.1,0.8),k=slider(.1,10,0.01,4.),y_0=slider(0,4,0.1,1.6),
                                  m=slider(0,4,0.1,1.),theta=slider(0,np.pi/2,0.1,np.pi/6))
    
    
output = interactive_plot.children[-1]
output.layout.height = '300px'
interactive_plot

interactive(children=(FloatSlider(value=-0.4, continuous_update=False, description='x_0a', max=6.0, min=-2.0),…

It follows then that we can define the rotated function as

In [13]:
def f_rotated(inputs,x_0a,x_0b,L,k,y_0,m,theta):
    yl,yu = -4,8
    buffer = 10
    num_points = len(inputs)
    outputs = speed_sigmoid_func(inputs,x_0a,x_0b,L,k,y_0,m)
    rot_mat = np.array([[np.cos(theta),-1.*np.sin(theta)],[np.sin(theta),np.cos(theta)]])
    rotation_origin = np.array([x_0a+(x_0b-x_0a)/2,y_0])
    rotation_origin_ones = np.repeat(rotation_origin[:,None],num_points,axis=1)
    inputs1,outputs1 = np.dot(rot_mat,np.vstack((inputs,outputs))-rotation_origin_ones)+rotation_origin_ones    
    which_inputs = find_nearest(inputs1,inputs[:,None])
    return outputs1[which_inputs]



In [14]:
def plot_f_rotated(x_0a,x_0b,L,k,y_0,m,theta):
    yl,yu = -4,8
    buffer = 10
    num_points = 1000
    inputs = np.linspace(yl-buffer,yu+buffer,num_points)
    outputs = f_rotated(inputs,x_0a,x_0b,L,k,y_0,m,theta)
    plt.figure(figsize=(8,8))
    ax = plt.subplot()
    ax.set_aspect('equal')
    plt.plot(inputs,outputs,'o',color='orange')
    plt.xlim([-10,10])
    plt.ylim([-10,10])
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')

    # Eliminate upper and right axes
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')

    # Show ticks in the left and lower axes only
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')



interactive_plot = interactive(plot_f_rotated, x_0a =slider(-2,6,0.1,-0.4),x_0b =slider(-2,6,0.1,3.6),
                                L=slider(0,4,0.1,0.8),k=slider(.1,10,0.01,4.),y_0=slider(0,4,0.1,1.6),
                                  m=slider(0,4,0.1,1.),theta=slider(0,np.pi/2,0.1,np.pi/6))
    
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(FloatSlider(value=-0.4, continuous_update=False, description='x_0a', max=6.0, min=-2.0),…

Now, replicate the above, and add in the un-rotated version with fixed parameters (the working version of the sigmoid function up till this point), and drag to find the parameters that best work for the rotated to match up with it in the left and right limit sections. 

In [15]:
def plot_f_rotated(x_0a,x_0b,L,k,y_0,m,theta):
    yl,yu = -4,8
    buffer = 10
    num_points = 1000
    inputs = np.linspace(yl-buffer,yu+buffer,num_points)
    
    plt.figure(figsize=(8,8))
    ax = plt.subplot()
    ax.set_aspect('equal')
    
    #The updating part of the plot is the (scatter) plot of the rotated function
    outputs = f_rotated(inputs,x_0a,x_0b,L,k,y_0,m,theta)
    plt.plot(inputs,outputs,'o',color='orange')
    
    #The fixed part is the non-rotated plot of the sigmoid with the previously determined parameters
    outputs1 = f_rotated(inputs,                     
                         x_0a = -0.4,
                         x_0b= 1.45,
                         L=0.8,
                         k=4.,
                         y_0=1.6,
                         m=1.,
                         theta=0.)
    
    plt.plot(inputs,outputs1,color='blue')
    
    
    plt.xlim([-10,10])
    plt.ylim([-10,10])
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')

    # Eliminate upper and right axes
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')

    # Show ticks in the left and lower axes only
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    ax.set_xticks(np.arange(-10,10,1))
    ax.set_yticks(np.arange(-10,10,1))



interactive_plot = interactive(plot_f_rotated, x_0a =slider(-2,6,0.01,-0.4),x_0b =slider(-2,6,0.01,1.45),
                                L=slider(0,4,0.1,0.8),k=slider(.1,10,0.01,4.),y_0=slider(0,4,0.1,1.6),
                                  m=slider(0,1,0.01,1.),theta=slider(0,np.pi/4,0.01,np.pi/6))
    
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(FloatSlider(value=-0.4, continuous_update=False, description='x_0a', max=6.0, min=-2.0, …

Final plot to check selected parameter values
x_0a = -0.4
x_0b = 1.45
L = 0.8
k = 2.4
y0 = 1.6
m = 0.43
theta = 0.37

In [16]:
def plot_f_rotated(x_0a,x_0b,L,k,y_0,m,theta):
    yl,yu = -4,8
    buffer = 10
    num_points = 1000
    inputs = np.linspace(yl-buffer,yu+buffer,num_points)
    
    plt.figure(figsize=(8,8))
    ax = plt.subplot()
    ax.set_aspect('equal')
    
    #The updating part of the plot is the (scatter) plot of the rotated function
    outputs = f_rotated(inputs,x_0a,x_0b,L,k,y_0,m,theta)
    plt.plot(inputs,outputs,'o',color='orange',label='Leaky Controller \'A\'')
    
    #The fixed part is the non-rotated plot of the sigmoid with the previously determined parameters
    outputs1 = f_rotated(inputs,                     
                         x_0a = -0.4,
                         x_0b= 1.45,
                         L=0.8,
                         k=4.,
                         y_0=1.6,
                         m=1.,
                         theta=0.)
    
    plt.plot(inputs,outputs1,color='blue',label='Perfect Controller')
    
    
    plt.xlim([-10,10])
    plt.ylim([-10,10])
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')

    # Eliminate upper and right axes
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')

    # Show ticks in the left and lower axes only
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    ax.set_xticks(np.arange(-10,10,1))
    ax.set_yticks(np.arange(-10,10,1))
    plt.legend()



interactive_plot = interactive(plot_f_rotated, x_0a =slider(-2,6,0.01,-0.4),x_0b =slider(-2,6,0.01,1.45),
                                L=slider(0,4,0.1,0.8),k=slider(.1,10,0.01,2.4),y_0=slider(0,4,0.1,1.6),
                                  m=slider(0,1,0.01,0.43),theta=slider(0,np.pi/4,0.01,0.37))
    
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(FloatSlider(value=-0.4, continuous_update=False, description='x_0a', max=6.0, min=-2.0, …

Now let's do the blue one as the first modified map we use in the direct arrival computations, and second try using the orange one.